# 4.2 Evaluating machine-learning models

In the three examples presented in chapter 3, we split the data into a **training set**, a **validation set**, and a **test set**. The reason not to evaludate the models on the same data  they were trained on quickly became evident: after just a few epochs, all three models began to **overfit**. That is, their performance on never-before-seen data started stalling (or worsening) compared to their performance on the training data - which always improves as training progresses. <br>
In machine learning, the goal is to achieve models that **generalize** - that perform well on never-before-seen data - and overfitting is the central obstacle. You can only control that which you can observe, so it's crucial to be able to reliably measure the generalization power of your model. The following sections look at strategies for mitigating overfitting and maximizing generalization. In this section, we'll focus on how to measure generalization : how to evaluate machine-learning models.